# Detection of Seismic Events using Bandpass Filter.

Bandpass filtering is used to clean the signal by removing noise, while matched filtering, using Signal-to-Noise Ratio (SNR) and template matching, detects events that resemble predefined seismic signatures. This improves the clarity and precision of event detection, particularly for faint signals.

## Install and import dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, correlate
import os

## Bandpass Implementation

In [ ]:
# Filters out frequencies outside the desired range to remove noise and retain relevant signals.
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs  # Nyquist frequency
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')  # Create filter coefficients
    filtered_data = filtfilt(b, a, data.fillna(0))  # Apply filter and handle missing values
    return filtered_data

In [ ]:
# This template is used for cross-correlation with real data.
def create_synthetic_template(sampling_rate=6.625, frequency=6.6, duration=20):
    t = np.linspace(0, duration, int(sampling_rate * duration))  # Time vector
    template = np.sin(2 * np.pi * frequency * t)  # Create sine wave
    return template

In [ ]:
# Finds similarity between the seismic data and the template to detect events.
def apply_matched_filter(data, template):
    cross_corr = correlate(data, template, mode='same')  # Compute cross-correlation
    threshold = 0.5 * np.max(cross_corr)  # Set threshold for event detection
    return cross_corr, threshold

In [ ]:
# Groups nearby events and extends their window to capture full event length.
def extend_event_detection(cross_correlation, threshold, extension=500, proximity=1000):
    detected_event_indices = np.where(cross_correlation > threshold)[0]
    extended_event_indices = []

    if len(detected_event_indices) > 0:
        start = detected_event_indices[0]
        end = start

        for i in range(1, len(detected_event_indices)):
            if detected_event_indices[i] - end <= extension:
                end = detected_event_indices[i]
            else:
                if extended_event_indices and start - extended_event_indices[-1][1] <= proximity:
                    extended_event_indices[-1] = (extended_event_indices[-1][0], end)
                else:
                    extended_event_indices.append((start, end))
                start = detected_event_indices[i]
                end = start

        # Save the final event
        extended_event_indices.append((start, end))

    return extended_event_indices

In [ ]:
# Function to plot detected events and filter out empty plots.
# If there are no detected events, no plot will be generated.
def plot_events(data, time_column, signal_column, events, file_name):
    if events:  # Only plot if there are detected events
        for idx, (start, end) in enumerate(events):
            plt.figure(figsize=(10, 4))
            plt.plot(data[time_column][start:end], data[signal_column][start:end], label="Event Signal")
            plt.title(f"Detected Event {idx+1} - {file_name}")
            plt.xlabel('Time (seconds)')
            plt.ylabel('Signal')
            plt.legend()
            plt.show()

In [ ]:
# Function to process the seismic data files.
# Each file is filtered, cross-correlated, and events are detected and plotted.
def model(test_dir, sampling_rate, lowcut=0.4, highcut=2.0):
    template = create_synthetic_template(sampling_rate, 6.6, 20)  # Create synthetic template
    
    for file_name in os.listdir(test_dir):
        if file_name.endswith(".csv"):  # Only process CSV files
            file_path = os.path.join(test_dir, file_name)
            data = pd.read_csv(file_path)  # Read data from file
            filtered_data = bandpass_filter(data['velocity(m/s)'], lowcut, highcut, sampling_rate)  # Filter data
            cross_corr, threshold = apply_matched_filter(filtered_data, template)  # Apply matched filter
            
            # Detect and extend event windows
            extended_event_windows = extend_event_detection(cross_corr, threshold)
            
            if extended_event_windows:  # Only plot if events are detected
                # Plot full signal with event regions highlighted
                plt.figure(figsize=(15, 6))
                plt.plot(data['time_rel(sec)'], cross_corr, label="Cross-correlation")
                for start, end in extended_event_windows:
                    plt.axvspan(data['time_rel(sec)'].iloc[start], data['time_rel(sec)'].iloc[end], color='red', alpha=0.5)
                
                plt.title(f"Matched Filter Detection - {file_name}")
                plt.xlabel('Time (seconds)')
                plt.ylabel('Cross-correlation')
                plt.legend()
                plt.show()
                
                # Plot each cropped event
                plot_events(data, 'time_rel(sec)', 'velocity(m/s)', extended_event_windows, file_name)
            else:
                print(f"No events detected in {file_name}, skipping plot.")

## Inference using Bandpass Filter

In [ ]:
if __name__ == "__main__": 
    test_dir = "./space_apps_2024_seismic_detection/data/lunar/test/data/S12_GradeA"  # Path to the directory with data files
    sampling_rate = 6.625
    print("Processing files in the test directory...")
    model(test_dir, sampling_rate)